<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/ContourExploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install ase

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 9.1 MB/s eta 0:00:00


In [2]:
import numpy as np
from ase.build import bulk
from ase.calculators.emt import EMT
from ase.md.contour_exploration import ContourExploration
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
bulk_Al_settings = {
    'maxstep': 1.0,
    'parallel_drift': 0.05,
    'remove_translation': True,
    'potentiostat_step_scale': None,
    'use_frenet_serret': True,
    'angle_limit': 20,
    'loginterval': 1}
size = 2
atoms = bulk('Al', 'fcc', cubic=True).repeat((size, size, size))
atoms.calc = EMT()

name = 'test_potentiostat'
seed = 19460926


E0 = atoms.get_potential_energy()

atoms.rattle(stdev=0.18, seed=seed)
initial_energy = atoms.get_potential_energy()

rng = np.random.RandomState(seed)
MaxwellBoltzmannDistribution(atoms, temperature_K=300, rng=rng)
with ContourExploration(
    atoms,
    **bulk_Al_settings,
    energy_target=initial_energy,
    rng=rng,
    trajectory=name + '.traj',
    logfile=name + '.log',
) as dyn:
    print("Energy Above Ground State: {: .4f} eV/atom".format(
        (initial_energy - E0) / len(atoms)))
    for _ in range(5):
        dyn.run(5)
        energy_error = (atoms.get_potential_energy() -
                        initial_energy) / len(atoms)
        print(f'Potentiostat Error {energy_error: .4f} eV/atom')
        # assert 0 == pytest.approx(energy_error, abs=0.01)

Energy Above Ground State:  0.1764 eV/atom
Potentiostat Error -0.0027 eV/atom
Potentiostat Error -0.0008 eV/atom
Potentiostat Error -0.0029 eV/atom
Potentiostat Error -0.0006 eV/atom
Potentiostat Error -0.0008 eV/atom
